In [1]:
from __future__ import print_function
import glob
import numpy as np
import math
from pyhdf.SD import SD, SDC
import matplotlib
#matplotlib.use('TkAgg')
import pylab
import matplotlib.pyplot as plt
import datetime

import subprocess
from numpy import genfromtxt
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly.graph_objs as go
from scipy.spatial import cKDTree
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import bs4
import itertools
import bottleneck as bn
from scipy.signal import lfilter
from itertools import compress
from numpy import inf
import pandas as pd

In [2]:
DATA = np.load('/home/kumarv/tayal007/MODIS_TILES/h08v05.npy')

In [39]:
def readModisfile(path):
    global mask
    file = SD(path, SDC.READ)
    sds_obj = file.select('Land_Cover_Type_1') 
    mask = np.array(sds_obj.get())
    return mask
def percentage(part, shape):
    whole = shape[0] * shape[1] * shape[2]
    return 100 * float(part)/float(whole)

def cleanNDVI(NDVISTACK):
	print ("Missing value in raw data is ", np.count_nonzero(np.isnan(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isnan(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Infinity value in raw data is ", np.count_nonzero(np.isinf(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isinf(NDVISTACK)),NDVISTACK.shape),'%')
	print ("Greater than 1 is ", np.count_nonzero(NDVISTACK > 1),'which is ' ,percentage(np.count_nonzero(NDVISTACK > 1),NDVISTACK.shape),'%')
	print ("Less than -1 is ", np.count_nonzero(NDVISTACK < -1),'which is ' ,percentage(np.count_nonzero(NDVISTACK < -1),NDVISTACK.shape),'%')
	
	CLEANED_DATA = np.copy(NDVISTACK)
	CLEANED_DATA[NDVISTACK == inf] = np.nan
	CLEANED_DATA[NDVISTACK == -inf] = np.nan
	CLEANED_DATA[NDVISTACK > 1] = np.nan
	CLEANED_DATA[NDVISTACK < -1] = np.nan

	print ("After Cleaning Missing value is ", np.count_nonzero(np.isnan(CLEANED_DATA)),'which is ' ,percentage(np.count_nonzero(np.isnan(CLEANED_DATA)),NDVISTACK.shape),'%')
	return CLEANED_DATA;
data2013 = np.copy(DATA[:,:,586:631])
data2013[data2013 < 0] = 0
data2013 = cleanNDVI(data2013)
#data2013 = data2013.reshape(data2013.shape[0] * data2013.shape[1], data2013.shape[2])
mask = readModisfile('/panfs/roc/groups/6/kumarv/tayal007/MODIS_yearly/MCD12Q1.A2013001.h08v05.051.2014308185137.hdf')
crop_data = data2013[mask==12]

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: RuntimeWarning:

invalid value encountered in less



Missing value in raw data is  16338907 which is  6.3035906635802466 %
Infinity value in raw data is  160282 which is  0.06183719135802469 %


/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: RuntimeWarning:

invalid value encountered in greater



Greater than 1 is  1946976 which is  0.7511481481481481 %


/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:15: RuntimeWarning:

invalid value encountered in less



Less than -1 is  0 which is  0.0 %


/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:20: RuntimeWarning:

invalid value encountered in greater

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:21: RuntimeWarning:

invalid value encountered in less



After Cleaning Missing value is  18285883 which is  7.054738811728395 %


In [44]:
row_mean = np.nanmean(crop_data, axis=1)
inds = np.where(np.isnan(crop_data))
crop_data[inds] = np.take(row_mean, inds[0])

In [45]:
a = crop_data[~np.isnan(crop_data).any(axis=1)]
print (a.shape)

(229337, 45)


In [47]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm

clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(crop_data)


OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [48]:
y_pred_train = clf.predict(crop_data)

In [50]:
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_train

22932

In [52]:
non_crop_data = data2013[mask!=12]
row_mean = np.nanmean(non_crop_data, axis=1)
inds = np.where(np.isnan(non_crop_data))
non_crop_data[inds] = np.take(row_mean, inds[0])
non_crop_data = non_crop_data[~np.isnan(non_crop_data).any(axis=1)]

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: RuntimeWarning:

Mean of empty slice



In [53]:
non_crop_data.shape

(5530662, 45)

In [54]:
y_pred_test = clf.predict(non_crop_data)
n_error_test = y_pred_test[y_pred_test == -1].size

In [55]:
n_error_test

3479313

In [56]:
n_error_test1 = y_pred_test[y_pred_test == 1].size
n_error_test1

2051349

In [60]:
non_crop_data = data2013[mask==5]
row_mean = np.nanmean(non_crop_data, axis=1)
inds = np.where(np.isnan(non_crop_data))
non_crop_data[inds] = np.take(row_mean, inds[0])
non_crop_data = non_crop_data[~np.isnan(non_crop_data).any(axis=1)]

In [62]:
print (non_crop_data.shape)
y_pred_test = clf.predict(non_crop_data)
n_error_test = y_pred_test[y_pred_test == -1].size

(18171, 45)


In [63]:
n_error_test

16135